# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config_geo import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [22]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,80.53,80,20,2.30,few clouds
1,1,Bereda,ES,43.2686,-7.5406,52.14,100,100,3.80,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,55.51,88,75,5.75,broken clouds
3,3,Khatanga,RU,71.9667,102.5000,-42.47,96,10,4.54,clear sky
4,4,Belmonte,BR,-15.8631,-38.8828,76.78,82,45,5.97,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [23]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? '))

print("Minimum temperature prefered is " + str(min_prompt_temp))
print("Maximum temperature prefered is " + str(max_prompt_temp))

Minimum temperature prefered is 70.0
Maximum temperature prefered is 85.0


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [24]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<=max_prompt_temp) & (city_data_df['Max Temp']>=min_prompt_temp)]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,80.53,80,20,2.30,few clouds
4,4,Belmonte,BR,-15.8631,-38.8828,76.78,82,45,5.97,scattered clouds
8,8,Rikitea,PF,-23.1203,-134.9692,77.09,72,100,1.92,light rain
10,10,Bambous Virieux,MU,-20.3428,57.7575,78.22,94,75,11.50,heavy intensity rain
12,12,Richards Bay,ZA,-28.7830,32.0377,77.31,88,66,4.65,broken clouds
15,15,Puerto Ayora,EC,-0.7393,-90.3518,77.56,90,57,4.00,broken clouds
16,16,Saint-Philippe,RE,-21.3585,55.7679,76.59,93,100,14.05,light rain
17,17,Bouna,CI,9.2667,-3.0000,75.88,19,63,3.15,broken clouds
21,21,Kirakira,SB,-10.4544,161.9205,82.90,80,100,9.84,overcast clouds
27,27,Hobyo,SO,5.3505,48.5268,76.05,78,46,21.68,scattered clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [27]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,80.53,80,20,2.30,few clouds
4,4,Belmonte,BR,-15.8631,-38.8828,76.78,82,45,5.97,scattered clouds
8,8,Rikitea,PF,-23.1203,-134.9692,77.09,72,100,1.92,light rain
10,10,Bambous Virieux,MU,-20.3428,57.7575,78.22,94,75,11.50,heavy intensity rain
12,12,Richards Bay,ZA,-28.7830,32.0377,77.31,88,66,4.65,broken clouds
15,15,Puerto Ayora,EC,-0.7393,-90.3518,77.56,90,57,4.00,broken clouds
16,16,Saint-Philippe,RE,-21.3585,55.7679,76.59,93,100,14.05,light rain
17,17,Bouna,CI,9.2667,-3.0000,75.88,19,63,3.15,broken clouds
21,21,Kirakira,SB,-10.4544,161.9205,82.90,80,100,9.84,overcast clouds
27,27,Hobyo,SO,5.3505,48.5268,76.05,78,46,21.68,scattered clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [28]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
0,Georgetown,MY,80.53,few clouds,5.4112,100.3354
4,Belmonte,BR,76.78,scattered clouds,-15.8631,-38.8828
8,Rikitea,PF,77.09,light rain,-23.1203,-134.9692
10,Bambous Virieux,MU,78.22,heavy intensity rain,-20.3428,57.7575
12,Richards Bay,ZA,77.31,broken clouds,-28.7830,32.0377


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [29]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
4,Belmonte,BR,76.78,scattered clouds,-15.8631,-38.8828,
8,Rikitea,PF,77.09,light rain,-23.1203,-134.9692,
10,Bambous Virieux,MU,78.22,heavy intensity rain,-20.3428,57.7575,
12,Richards Bay,ZA,77.31,broken clouds,-28.7830,32.0377,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [31]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [43]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
Georgetown - nearest hotel: Page 63 hostel
Belmonte - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Bambous Virieux - nearest hotel: No hotel found
Richards Bay - nearest hotel: Protea Hotel
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Saint-Philippe - nearest hotel: Le Baril
Bouna - nearest hotel: Hôtel Éléphant  Bouna
Kirakira - nearest hotel: No hotel found
Hobyo - nearest hotel: No hotel found
Atuona - nearest hotel: Pearl Resort
Hithadhoo - nearest hotel: Pebbles Inn
Marsh Harbour - nearest hotel: Lofty Fig Villas
Hualmay - nearest hotel: Hotel Mi Choacan
Mbandaka - nearest hotel: Hôtel Epervier
Santa Isabel - nearest hotel: No hotel found
Namibe - nearest hotel: Hotel Chik Chik Namibe
Mar Del Plata - nearest hotel: Nuevo Ostende
Sechura - nearest hotel: Hospedaje Claritza
Kraksaan - nearest hotel: No hotel found
Waingapu - nearest hotel: HOTEL SANDLEWOOD
Thinadhoo - nearest hotel: No hotel found
Tagusao - n

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Page 63 hostel
4,Belmonte,BR,76.78,scattered clouds,-15.8631,-38.8828,No hotel found
8,Rikitea,PF,77.09,light rain,-23.1203,-134.9692,Chez Bianca & Benoit
10,Bambous Virieux,MU,78.22,heavy intensity rain,-20.3428,57.7575,No hotel found
12,Richards Bay,ZA,77.31,broken clouds,-28.7830,32.0377,Protea Hotel


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [52]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"].str.contains("No hotel found") == False]

# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Page 63 hostel
8,Rikitea,PF,77.09,light rain,-23.1203,-134.9692,Chez Bianca & Benoit
12,Richards Bay,ZA,77.31,broken clouds,-28.7830,32.0377,Protea Hotel
15,Puerto Ayora,EC,77.56,broken clouds,-0.7393,-90.3518,Hostal La Mirada De Solitario George
16,Saint-Philippe,RE,76.59,light rain,-21.3585,55.7679,Le Baril


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [54]:
# Create the output File (CSV)
output_search_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_search_file, index_label='City_ID')

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [57]:
# Configure the map plot
hotel_map_plot = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City")

# Display the map
hotel_map_plot

ImportError: Geographic projection support requires GeoViews and cartopy.